In [17]:
!pip install transformers datasets
!pip install bs4

In [18]:
import pandas as pd
from datasets import load_dataset
ds = load_dataset("GonzaloA/fake_news", split="train")
df = ds.to_pandas()


Repo card metadata block was not found. Setting CardData to empty.


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report

df = df.dropna(subset=['text', 'label'])
X = df['text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
tfidf_vectorizer = TfidfVectorizer(max_features=2500, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [22]:
from transformers import pipeline
classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english", truncation=True)
sample_size = 50
X_sample = X_test.head(sample_size).tolist()
y_sample = y_test.head(sample_size).tolist()
y_pred_raw = classifier(X_sample)
y_pred = [1 if p['label'] == 'POSITIVE' else 0 for p in y_pred_raw]
accuracy = accuracy_score(y_sample, y_pred)
report = classification_report(y_sample, y_pred, target_names=['True', 'Fake'])

Device set to use cpu


In [24]:
import requests
from bs4 import BeautifulSoup
def predict_article(url):
  try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    text = soup.get_text()
    #data is truncated for token reasons
    text_to_predict = text[:512]
    prediction_raw = classifier(text_to_predict)
    prediction = prediction_raw[0]['label']
    return "Real" if prediction == 'POSITIVE' else "Fake"
  except requests.exceptions.RequestException as e:
    return f"Error: Could not retrieve article from URL. {e}"
  except Exception as e:
    return f"Error: An unexpected error occurred. {e}"

In [25]:
url_to_check = input("Please enter a URL to check: ")
prediction = predict_article(url_to_check)
print(f"The article at {url_to_check} is predicted to be {prediction}.")

Please enter a URL to check: https://abcnews.go.com/Politics/trump-calls-epstein-files-irrelevant-push-release-gains/story?id=125225706
The article at https://abcnews.go.com/Politics/trump-calls-epstein-files-irrelevant-push-release-gains/story?id=125225706 is predicted to be Fake.
